<a target="_blank" href="https://colab.research.google.com/github/bettercodepaul/machine_learning_workshop/blob/main/Predictive_Analytics_Einführung.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import urllib.request
import os.path

import polars as pl
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
# download CSV data
DATA_URL = "https://raw.githubusercontent.com/bettercodepaul/machine_learning_workshop/main/AmesHousing.csv"
LOCAL_DATA_FILE_NAME = os.path.basename(DATA_URL)
urllib.request.urlretrieve(DATA_URL, LOCAL_DATA_FILE_NAME)

In [ ]:
# download excercises and utility functions
EXERCISES_URL = "https://raw.githubusercontent.com/bettercodepaul/machine_learning_workshop/main/exercises.py"
urllib.request.urlretrieve(EXERCISES_URL, os.path.basename(EXERCISES_URL))

In [ ]:
from exercises import *

# Verkaufspreise von Häusern vorhersagen

Zuerst lesen wir die Daten ein und machen ein wenig EDA.

Mit der Funktion `read_csv` von Pandas können wir die Daten einlesen.

In [ ]:
df = pl.read_csv("AmesHousing.csv")

In [ ]:
# die Funktion head gibt die ersten fünf Datensätze aus
df.head()

In [ ]:
# die Funktion describe gibt ein paar statistische Infos je Spalte aus
df.describe()

Wir haben 9 unterschiedliche Informationen über die Hausverkäufe:

* Nachbarschaft: In welcher Nachbarschaft steht das Haus?
* Qualität: Wie ist die Qualität der Materialien des Hauses?
* Zustand: Wie ist der Zustand des Hauses?
* Wohnflaeche: Wie groß ist die Wohnfläche in Quadratfuß?
* Baujahr: Wann wurde das Haus gebaut?
* Verkaufsjahr: Wann wurde das Haus verkauft?
* Preis: Für wie viel US-Dollar wurde das Haus verkauft?

Mit der Funktion `scatter` von Plotly kannst Du einen Scatterplot zeichnen, der gut ist um die Zusammenhänge zwischen Variablen zu erkennen. Betrachte den Zusammenhang zwischen den unterschiedlichen Variablen und der Zielvariable `Preis`. Gibt es Ausreißer bei der Wohnfläche?

In [ ]:
px.scatter(df, x="Wohnflaeche", y="Preis", opacity=0.25)

## Ausreißer entfernen

Wir können die Daten mit `filter` und einem logischen Ausdruck einschränken.

In [ ]:
# alle Häuser mit einem Preis größer als (gt=greater than) 700.000 US$
expensive_houses = df.filter(pl.col("Preis").gt(700000))
expensive_houses

In [ ]:
# alle Häuser mit einem Preis kleiner als (le=less or equal) 50.000 US$
cheap_houses = df.filter(pl.col("Preis").le(30000))
cheap_houses

Für die praktischen Übungen haben wir ein simples Quizsystem.

In [ ]:
# Frage ausgeben
q0.question()

In [ ]:
# Lösung angeben
solution = ...

In [ ]:
# Lösung prüfen
q0.check(solution)

In [ ]:
# Hinweis auf die Lösung bekommen
q0.hint()

In [ ]:
# Lösung anzeigen
q0.solution()

## Übung 1: Ausreißer entfernen
Es gibt ein paar Häuser mit sehr hoher Wohnfläche. Entferne alle Häuser mit mehr als 4000 Quadratfuß Wohnfläche.

In [ ]:
q1.question()

In [ ]:
q1_df = ...

In [ ]:
q1.check(q1_df)

# Trainings und Testdatensatz erstellen

Wir teilen unsere Daten in einen Trainings und einen Testdatensatz auf. Die meisten ML-Algorithmen mögen nur numerische Spalten, deshalb verwenden wir die Spalte `Nachbarschaft` jetzt noch nicht.

In [ ]:
X = df.drop(columns=["Preis", "Nachbarschaft"])
y = df.get_column("Preis")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Übung 2: Train und Test vergleichen
Untersuche die Datensätze `X_train`, `y_train` und `X_test`, `y_test` darauf, ob sie ähnliche statistische Eigenschaften haben.

In [ ]:
q2.question()

In [ ]:
X_train_Zustand_median = ...
y_train_median = ...
X_test_Zustand_median = ...
y_test_median = ...

In [ ]:
q2.check(X_train_Zustand_median, y_train_median, X_test_Zustand_median, y_test_median)

# Lineares Modell trainieren

Jetzt können wir schon unser erstes Modell trainieren. Wir trainieren das lineare Modell `sklearn.linear_model.LinearRegression` mit der Methode `fit`.

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

for feature, coefficient in zip(model.feature_names_in_, model.coef_):
    print(f"{feature}: {coefficient:.1f}")
print(f"Intercept: {model.intercept_:.1f}")

## Vorhersagen treffen und validieren

Mit der Methode `predict` können wir Vorhersagen treffen.

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
idx = 10
print(f"Vorhergesagter Preis: {y_pred[idx]:.1f}")
print(f"Wirklicher Preis: {y_test[idx]:.1f}")
print(f"Abweichung: {(y_pred[idx] - y_test[idx])/y_test[idx]:.1%}")
X_test[idx]

Wie gut die Vorhersage im Mittel ist, können wir mit dem Bestimmtheitsmaß $R^2$ messen. Für eine perfekte Vorhersage gilt $R^2=1$. Für eine konstante Vorhersage mit dem  Mittelwert gilt $R^2=0$.

In [ ]:
# R^2 für eine perfekte Vorhersage ist 1.0
y_pred_perfect = y_test
metrics.r2_score(y_test, y_pred_perfect)

In [ ]:
# R^2 für eine Vorhersage, die immer den Mittelwert vorhersagt, ist 0.0
y_pred_mean = [y_test.mean()]*len(y_test)
metrics.r2_score(y_test, y_pred_mean)

In [ ]:
# R^2 = 0.75 kann gelesen werden als: 75% der Varianz in den Daten werden durch das Modell erklärt
metrics.r2_score(y_test, y_pred)

## Übung 3: Modell verbessern mit weniger Features

Wie verändert sich die Qualität der Vorhersage, wenn Du zusätzliche Features entfernst (also nicht nur `Nachbarschaft`)?

In [ ]:
q3.question()

In [ ]:
X = df.drop(columns=["Preis", "Nachbarschaft"]) # hier kannst Du zusätzliche Features entfernen
y = df.get_column("Preis")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.r2_score(y_test, y_pred)

In [ ]:
q3.check(y_test, y_pred)

## Modell verbessern mit zusätzlichen Features

Wir können zusätzliche Features berechnen und so unserem Modell helfen bessere Vorhersagen zu machen.

Zum Beispiel könnten wir die Qualität mit dem Zustand multiplizieren.

In [ ]:
df = pl.read_csv("AmesHousing.csv")
df = df.with_columns(
    (pl.col("Zustand")*pl.col("Qualitaet")).alias("Qualitaet*Zustand")
)
df.head()

In [ ]:
X = df.drop(columns=["Preis", "Nachbarschaft"])
y = df.get_column("Preis")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.r2_score(y_test, y_pred)

## Übung 4: Zusätzliches Features Alter beim Verkauf

In [ ]:
q4.question()

In [ ]:
# Daten neu laden und R2 ohne neues Feature berechnen -> diesen Code NICHT ändern!
df_before = pl.read_csv("AmesHousing.csv")
model_before, r2_before = train_fit_predict_score(LinearRegression(), df_before)
print(r2_before)

In [ ]:
# hier das neue Feature hinzufügen -> diesen Code ERGÄNZEN!
df_after = pl.read_csv("AmesHousing.csv")
df_after = ...

In [ ]:
# R2 mit neuem Feature berechnen -> diesen Code NICHT ändern!
if type(df_after) != type(Ellipsis):
    model_after, r2_after = train_fit_predict_score(LinearRegression(), df_after)
    print(r2_after)
    if r2_after > r2_before:
        print("Das neue Feature hat die Vorhersage verbessert!")
    elif r2_after < r2_before:
        print("Das neue Feature hat die Vorhersage verschlechtert!")
    else:
        print("Das neue Feature hat die Vorhersage nicht verändert!")
else:
    r2_after = None

In [ ]:
q4.check(df_after, r2_before, r2_after)

## Andere Modelle (Zusatzaufgabe 1)

In [ ]:
q5.question()

In [ ]:
q5_model = ...

In [ ]:
q5.check(q5_model)

## Information über die Nachbarschaften nutzen (Zusatzaufgabe 2)

Jetzt wollen wir die Spalte `Nachbarschaft` nutzen. Wir können die Spalte einfach in einen numerischen Index-Wert umwandeln (1, 2, 3, ...) und schauen, ob das den Modellen hilft. Das nennt man auch *Integer Encoding*.

In [ ]:
df_int_encoding = df.with_columns(pl.col("Nachbarschaft").cast(pl.Categorical).cast(pl.Int64).alias("NachbarschaftIntegerEncoding"))
df_int_encoding.sample(5)

Eine andere Variante ist es für jeden Wert der Spalte eine eigene Spalte anzulegen, die ja nach Ausprägung jeweils den Wert `0` oder `1` annimmt. Das nennt man *One Hot Encoding* oder auch *Dummy Encoding*.

In [ ]:
df_dummy_encoding = df.to_dummies("Nachbarschaft")
df_dummy_encoding.sample(5)

Die dritte Variante ist das *Target Encoding*. Wir nutzen die Infos aus den Trainingsdaten und berechnen den durchschnittlichen Verkaufspreis je Nachbarschaft. Wichtig zu beachten ist, dass wir dafür nur den Trainingsdatensatz benutzen.

In [ ]:
X = df.drop(columns="Preis")
y = df.get_column("Preis")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

target_encoding = X_train.hstack(pl.DataFrame(y_train)).group_by("Nachbarschaft").agg(pl.col("Preis").mean().alias("NachbarschaftsPreis"))
target_encoding.sample(5)

In [ ]:
q6.question()

In [ ]:
q6_best_encoding_linear_regression = ... # "Target Encoding", "Dummy Encoding" oder "Integer Encoding"
q6_best_encoding_random_forest = ... # "Target Encoding", "Dummy Encoding" oder "Integer Encoding"# 

In [ ]:
q6.check(q6_best_encoding_linear_regression, q6_best_encoding_random_forest)